# Logistic Regression

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
# Importing the dataset
df = pd.read_csv('kr_2.csv', sep=';')
df.head(20)

,RetailSpace,Type,NumberOfReviewInGoogle,The_average_number_of_units_in_the_check,human_traffic,AverageIncomeOfThePopulation,Stop_GT,The_final_stop_of_the_GT,Crosswalk,Specialized_bread,Success
0,0,0,42.820513,0,5211.000000,15610.000000,1,0,0,1,0
1,1,0,8.000000,1,2424.000000,12600.000000,1,1,0,0,0
2,2,1,42.820513,2,3868.000000,12600.000000,1,0,1,1,0
3,3,1,42.820513,3,4546.000000,13930.000000,1,0,0,1,0
4,1,1,42.820513,4,3244.000000,16030.000000,0,0,0,0,0
5,4,0,9.000000,5,2560.000000,12320.000000,0,0,0,1,0
6,5,0,21.000000,6,1842.000000,16100.000000,0,0,1,0,1
7,6,0,227.000000,7,412.000000,12180.000000,0,0,0,1,1
8,7,0,42.820513,8,233.000000,13930.000000,0,0,0,0,0
9,8,0,17.000000,9,978.000000,14560.000000,0,0,1,0,0


Загружен датасет, который был ранее обработан. Итак, зависимая бинарная переменая в моделе - success, которая показывает уровень успешности магазина.

Цель классификации была поставлена ранее.

Cледует заметить, что модель логистической регрессии будет выступать базовой. На ее основе отберем значимые для моделирования переменнные и отталкиваясь от которой будет принято решение об целесообразности более сложных классификаторов.



In [3]:
df.shape

(69, 11)

In [4]:
# Splitting the dataset into the Training set and Test set
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

Разделим датасет на множество обьяснемой переменной (success) и обьясняющих переменных. Причем отобрано по итогу последующих экспериментов такой размер тестовой выборки, когда она наиболее сбалансирована и получено максимальное качество модели.

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler().fit(X_train)
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)

Для корректной работы данные были отшкалированы.

# FEATURE SELECTION

In [6]:
#!pip install statsmodels 

In [7]:
# Baseline model
import statsmodels.api as sm
lr = sm.Logit(y_train, X_train).fit()
print(lr.summary2())

Optimization terminated successfully.
         Current function value: 0.599175
         Iterations 6
                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.093   
Dependent Variable: y                AIC:              81.1159 
Date:               2021-12-02 13:57 BIC:              100.4341
No. Observations:   51               Log-Likelihood:   -30.558 
Df Model:           9                LL-Null:          -33.675 
Df Residuals:       41               LLR p-value:      0.71623 
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.5964     0.3762    1.5854   0.1129   -0.1409   1.3337
x2        0.3895     0.3277    1.1886   0.2346   -0.2528   1.0318
x3        0.6305 

Построена базовая модель, которая помогает отобрать значимые переменные. Но для более точного анализа воспользуемся автоматическим выбором переменных.

In [8]:
# Automatic Feature Selection
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
estimator=LogisticRegression(random_state=123)
selector=RFECV(estimator, step=1)
selector=selector.fit(X_train, y_train)
print(selector.n_features_)
print(selector.support_)
print(selector.ranking_)

8
[ True  True  True False  True  True  True False  True  True]
[1 1 1 3 1 1 1 2 1 1]


Автоматический отбор показывает, что в моделе 8 значимых переменных. Итак, дальнейшая модель будет построена на них.

In [9]:
# Selected Features
selected_columns=[]
for i in range(len(X_train[0])):
    if selector.support_[i]==1:
        selected_columns.append(i)
print(selected_columns)
print('№\tIndex\tFeature')
i=0
for column in selected_columns:
    print(str(i)+'\t'+str(column)+'\t'+str(df.columns[column]))
    i=i+1

[0, 1, 2, 4, 5, 6, 8, 9]
№	Index	Feature
0	0	RetailSpace
1	1	Type
2	2	NumberOfReviewInGoogle
3	4	human_traffic
4	5	AverageIncomeOfThePopulation
5	6	Stop_GT
6	8	Crosswalk
7	9	Specialized_bread


Выведены наиболее значимые переменные.

Итак:
  
  1.RetailSpace - по статистике, супермаркеты с большей площадью действительно привлекают большее число клиентов (разнообразным выбором, малой вероятностью очереди и т.д.), поэтому и шанс на успешность выше
  
  2.type - тип магазина - смежное понятие с площадью и также является индикатором для лучше оценки
   
  3.NumberOfReviewInGoogle - естественно, как никогда является акутальным отзывы об магазине в интернете. Каждый покупатель может сформировать свое мнение еще до того, как попадет в сам магазин. Поэтому данная метрика напрямую показывает успех магазина
 
  4.human_traffic - чем больше поток людей в магазине, тем лучше сам магазин. Значит человеку нравится качество продуктов, ассортимент и отсюда большее число людей посещает магазин. А собствеено и успешность выше.
    
  5.Stop_GT - естественно, что остановка рядом с магазино делаем его более успешным. Вряд ли человек будет выбирать магазин до кторого идти 20 минут с остановки чтобы закупить 2-3 пакеты еды.
    
   6.Crosswalk - интересный и неожиданный показатель, который влияет на успешность магазина. Но, можно предположить, что это так же является показателем успеха. В основном перед входом в огромные супермаркеты обильный поток машин, стоянки и простому пешеходу сложно безпрепятсвенно добраться до входа. Поэтому наличие пешеходного  облегчает эту задачу.
   
   7. AverageIncomeOfThePopulation- средний доход потребителя зависит от того насколько масштабные будут покупки в магазине.
   
   8. Specialized_bread - личная выпечка у магазина показывает его уровень, а соответсвенно и является показателем того, что магазин будет более успешный.

In [10]:
# Leaving only Selected Features
X_train_select=X_train[:, selected_columns]
X_test_select=X_test[:, selected_columns]

In [11]:
# p-value < 1% Features
X_train = X_train[:,[True, True, True, False, True, True, True, False, True, True]]
X_test = X_test[:,[True, True, True, False, True, True, True, False, True, True]]

Модель будет построена на 8 переменных, которые наиболее значимы по результатам исследования.

In [12]:
# Fitting Logistic Regression to the Training set 
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 13).fit(X_train, y_train)

In [13]:
# Predicting the Test set results
y_pred = lr.predict(X_test)
lr.score(X_test,y_test)

0.8333333333333334

Получено довольно высокое качество модели. Примерно 83% обьектов тестовой выборки распознаны верно.

In [14]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[11  1]
 [ 2  4]]


Построена таблица сопряженности на основе тестовой выборки. 15 примеров распознаны верно, неверно распознано 3. Таким образом 2 обьектов модель отнесла неверно к классу False и, соответсвенно, 1 обьект не правильно отнесены к классу True.

Итак, магазины, которые действительно отнесены к классу успешных, но модель их распознала неверно - 2, а наоборот - таких 1.

Но логично предположить, что исходя из поставленной задачи ранее мы не стремимся с большей вероятностью определить либо успешный это магазин, либо неуспешный. Нужно найти лучшую модель, которая сможет наилучшим образом определить к какому классу отнесется магазин и исходя из этого уже выдвигать какие-то рекомендации по улучшению, маркетинговой рекламе и тд (либо оставить все как есть, если магазин итак уже успешен)

Поэтому в дальнейших исследованиях будем делать акцент на качестве моделе и верной распознаваемости выборки.

In [15]:
import pickle
obj = {'X_train': X_train, 'X_test': X_test,'y_train': y_train,'y_test': y_test}
output = open('data_LogRegr.pkl', 'wb')
pickle.dump(obj, output, 2)
output.close()